## Handoffs

### A way for an agent to invoke another agent

In [19]:
from agents import Runner, Agent
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [38]:
from agents import Agent, Runner, handoff
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from pydantic import BaseModel

class Outline(BaseModel):
    outline: str

class Tutorial(BaseModel):
    outline: str
    tutorial: str

tutorial_generator = Agent(
    name="Tutorial Generator",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    Expand the supplied outline into a full Markdown tutorial with code snippets.""",
    output_type=Tutorial,
)

outline_builder = Agent(
    name="Outline Builder",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    Create an outline for the requested topic, *then call ⁠ transfer_to_tutorial_generator ⁠*.
    Do *not* write the tutorial yourself.""",
    handoffs=[handoff(tutorial_generator, input_type=Outline, on_handoff=lambda ctx, data: None,   #required when input_type is set 
                     )],
    output_type=Outline,   # stops the builder from “finishing” the tutorial
)

In [41]:
tutorial_response = await Runner.run(outline_builder, "Loops in Assembly")
print(tutorial_response.final_output)

outline='### Introduction to Loops in Assembly Language\n- Definition and importance of loops in programming\n- Differences between high-level and assembly language loops\n\n### Basic Loop Structure in Assembly\n- Overview of assembly language syntax\n- Explanation of registers and their roles\n\n### Types of Loops in Assembly\n- **Counter-Controlled Loops**\n  - Overview\n  - Example with code snippet\n- **Condition-Controlled Loops**\n  - Overview\n  - Example with code snippet\n\n### Implementing Loops\n- **For Loop Equivalent**\n  - Initialization of registers\n  - Loop condition and increment\n  - Code example\n- **While Loop Equivalent**\n  - Setting initial conditions\n  - Testing the condition\n  - Code example\n- **Do-While Loop Equivalent**\n  - Execution and conditional check\n  - Code example\n\n### Managing Loop Execution\n- Using flags or sentinel values\n- Jump instructions (JMP, JE, JNE, etc.)\n- Loop unwinding for optimization\n\n### Practical Examples\n- Simple number

In [42]:
print(tutorial_response.final_output.outline)

### Introduction to Loops in Assembly Language
- Definition and importance of loops in programming
- Differences between high-level and assembly language loops

### Basic Loop Structure in Assembly
- Overview of assembly language syntax
- Explanation of registers and their roles

### Types of Loops in Assembly
- **Counter-Controlled Loops**
  - Overview
  - Example with code snippet
- **Condition-Controlled Loops**
  - Overview
  - Example with code snippet

### Implementing Loops
- **For Loop Equivalent**
  - Initialization of registers
  - Loop condition and increment
  - Code example
- **While Loop Equivalent**
  - Setting initial conditions
  - Testing the condition
  - Code example
- **Do-While Loop Equivalent**
  - Execution and conditional check
  - Code example

### Managing Loop Execution
- Using flags or sentinel values
- Jump instructions (JMP, JE, JNE, etc.)
- Loop unwinding for optimization

### Practical Examples
- Simple number counter loop
- Array iteration in assembly


In [43]:
print(tutorial_response.final_output.tutorial)

## Introduction to Loops in Assembly Language

Loops are fundamental constructs in programming that allow you to repeat a block of code multiple times. In assembly language, loops are implemented using jump instructions to repeat execution until a condition is met, differing significantly from high-level languages that offer built-in loop constructs like `for`, `while`, and `do-while`.

## Basic Loop Structure in Assembly

Assembly language use registers to control loop conditions and counter variables. Registers are small storage locations within the CPU that hold data and addresses. Depending on the architecture (e.g., x86, ARM), different registers are available.

## Types of Loops in Assembly

### Counter-Controlled Loops
Counter-controlled loops repeat a block of code a specific number of times. 

**Example:**
```assembly
section .data
    counter db 10    ; loop counter initialized to 10

section .text
global _start

_start:
    mov ecx, dword [counter] ; load counter into ecx re

In [16]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name = "History Tutor",
    handoff_description= "Specialist tutorial for historical questions.",
    instructions= "You provide assistance with historical queries. Explain important events and contect clearly"
)

math_tutor_agent = Agent(
    name = "Math Tutor",
    handoff_description= "Specialist tutorial for math questions.",
    instructions= "You provide assistance with math queries. Explain your reasoning at each step and include examples."
)

def on_history_handoff(ctx : RunContextWrapper[None]):
    print("Handing off to history tutor agent")

def on_math_handoff(ctx : RunContextWrapper[None]):
    print("Handing off to math tutor agent")

triage_agent = Agent(
    name= "Triage Agent",
    instructions= "You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs= [handoff(history_tutor_agent, on_handoff=on_history_handoff), 
               handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

In [17]:
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
print(result.final_output)

Handing off to math tutor agent
Adding 2 and 2 is a simple arithmetic problem. Here's a step-by-step explanation:

1. **Understand the Operation**: Addition is combining two or more numbers to get a total.

2. **Identify the Numbers**: We have two numbers here: 2 and 2.

3. **Perform the Addition**:
   - Write the numbers in a column if that helps. For example:\n
     \n   2\n   + 2\n   ---\n
   - Add the numbers in the column. 

4. **Calculate**: 
   - 2 plus 2 equals 4. 
   - So, 2 + 2 = 4.

5. **Conclusion**: 
   - The sum of 2 and 2 is 4.

Example:
If you have 2 apples and someone gives you 2 more apples, you’ll have 4 apples in total.

This straightforward calculation demonstrates basic addition.


In [18]:
result = await Runner.run(triage_agent, "How did WW2 start?")
print(result.final_output)

Handing off to history tutor agent
World War II officially began on September 1, 1939, when Germany, led by Adolf Hitler, invaded Poland. This invasion prompted Britain and France to declare war on Germany two days later, on September 3. Here are some key factors that led to the outbreak of the war:

1. **Treaty of Versailles**: The harsh terms of this treaty, imposed on Germany after World War I, created economic hardships and widespread resentment among Germans. This discontent was exploited by Hitler to gain support.

2. **Expansionist Policies**: Hitler's aggressive expansionist policies aimed to unite all German-speaking people and gain living space (Lebensraum) for Germany. He had already annexed Austria in 1938 and, later that year, parts of Czechoslovakia.

3. **Failure of Appeasement**: European powers, primarily Britain and France, initially followed a policy of appeasement, allowing Hitler to expand German territory without facing significant opposition, hoping to avoid anot

In [50]:
from agents import function_tool
from pydantic import BaseModel

class ManagerEscalation(BaseModel):
    issue : str
    why : str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue : str):
    """
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

def on_manager_handoff(ctx : RunContextWrapper[None], input= ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference
    
manager_agent = Agent(
    name = "Manager Agent",
    handoff_description= "Handles escalated issues that require managerial attention",
    instructions= (
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."),
    tools = [create_ticket]
)

customer_service_agent = Agent(
    name = "Customer Service",
    instructions= "You assist customers with general inquiries and basic troubleshooting. " + 
                  "If the issue cannot be resolved, escalte it to the manager along with the reason why you cannot fox the issue yourself.",
    handoffs=[handoff(manager_agent, input_type= ManagerEscalation, on_handoff= on_manager_handoff)]
)

In [51]:
result = await Runner.run(customer_service_agent, "I want a refund, but your system won't let me process it. The website is just blank for me")
print(result.final_output)

Escalating to manager agent:  Customer cannot process a refund through the website because the page is blank.
Reason for escalation:  Customer is experiencing a technical issue with the website displaying a blank page. This is beyond basic troubleshooting capabilities and requires managerial intervention.
Creating ticket for issue: Customer cannot process a refund through the website because the page is blank. Needs technical team intervention to resolve the issue.
I've created a ticket for your issue with our technical team. The ticket ID is 12345. They'll look into this and work on resolving the issue. Thank you for your patience! If you need further assistance, feel free to ask.
